# CS345 - CNN on CIFAR10 - Keras Functional Model




# Preparation of Runtime environment
Mount Google Drive and check that the proper folder exists under CS345

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/CS345/CS345_Project2"

Mounted at /content/gdrive/
--------------------------------------------------------------------------------
'CNN -CIFAR10.ipynb'   data  'Image Data.ipynb'   lib   models	 rx   testSet


## Begin of source code
Use python to change the directory to the root folder of this project. This is required in order to import packages that are subfolders in the project.


In [ ]:
import os

# Change to this source code folder
os.chdir("/content/gdrive/My Drive/Colab Notebooks/CS345/CS345_Project2")
print("Current dir: ", os.getcwd())

Current dir:  /content/gdrive/My Drive/Colab Notebooks/CS345/CS345_Project2


In [ ]:
import os
import tensorflow as tf
import numpy as np
from rx.visualization import CPlot
from tensorflow import keras

from data.cifar10 import CCIFAR10DataSet
from lib import CEvaluator
from sklearn import preprocessing

from tensorflow.keras.layers import Input, Flatten, Dense, Activation, Softmax
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# What tensorflow version is installed in this VM?
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


Create the dataset object that loads all data into the memory. Create the one-hot vectors as targets for classification

In [ ]:
# ... // Create the data objects \\ ...
oDataset = CCIFAR10DataSet()

# One hot encoding for the training and validation set labels
nTSLabelsOnehot = keras.utils.to_categorical(oDataset.TSLabels)
nVSLabelsOnehot = keras.utils.to_categorical(oDataset.VSLabels)

- Download progress: 100.0%
Download finished. Extracting files.
Done.
Classes: 10


Declare the class of the model that contains the architecture and functions needed to train it

In [ ]:
# =========================================================================================================================
class CCNNSimple(object): 
  # --------------------------------------------------------------------------------------
  def __init__(self, p_oInputShape=[32,32,3], p_nModuleCount=6, p_bHasBias=True):
    # ................................................................
    self.InputShape   = p_oInputShape
    self.ModuleCount  = p_nModuleCount
    self.HasBias      = p_bHasBias
    self.Input        = None
    self.Modules      = [None]*self.ModuleCount
    self.Output       = None
    self.OutputTensor = None
    self.Model        = None
    # ................................................................
    self.DefineModel()
  # --------------------------------------------------------------------------------------  
  def DefineModel(self):

    self.Input = Input(shape=self.InputShape) 
    tA = self.Input
    
    # // Module 1 \\
    tA = Conv2D(96, kernel_size=3, strides=2, padding="same",  
                  kernel_initializer="glorot_uniform", use_bias=self.HasBias, bias_initializer="zeros")(tA)
    tA = Activation("relu")(tA)
    tA = MaxPooling2D(pool_size=[2, 2], strides=[1, 1], padding="same")(tA) 
    tA = BatchNormalization()(tA)
    
    # // Module 2 \\
    tA = Conv2D(256, kernel_size=3, strides=1, padding="same",  
                  kernel_initializer="glorot_uniform", use_bias=self.HasBias, bias_initializer="zeros")(tA)
    tA = Activation("relu")(tA)
    tA = MaxPooling2D(pool_size=[2, 2], strides=[2, 2], padding="same")(tA)
    tA = BatchNormalization()(tA)

    # // Module 3 \\
    tA = Conv2D(384, kernel_size=3, strides=1, padding="same",  
                  kernel_initializer="glorot_uniform", use_bias=self.HasBias, bias_initializer="zeros")(tA)
    tA = Activation("relu")(tA)
    tA = BatchNormalization()(tA)

    # // Module 4 \\
    tA = Conv2D(384, kernel_size=3, strides=1, padding="same",  
                  kernel_initializer="glorot_uniform", use_bias=self.HasBias, bias_initializer="zeros")(tA)
    tA = Activation("relu")(tA)
    tA = BatchNormalization()(tA)

    # // Module 5 \\
    tA = Conv2D(256, kernel_size=3, strides=1, padding="same",  
                  kernel_initializer="glorot_uniform", use_bias=self.HasBias, bias_initializer="zeros")(tA)
    tA = Activation("relu")(tA)
    tA = MaxPooling2D(pool_size=[2, 2], strides=[2, 2], padding="same")(tA) 
    tA = BatchNormalization()(tA)
    

    # // Module 6 - Classifier\\
    tA = Flatten()(tA)
    
    tA = Dropout(0.5)(tA) 
    tA = Dense(1024, use_bias=self.HasBias, kernel_initializer="glorot_uniform", bias_initializer="zeros")(tA)
    tA = Activation("relu")(tA)

    tA = Dropout(0.5)(tA)
    tA = Dense(1024, use_bias=self.HasBias, kernel_initializer="glorot_uniform", bias_initializer="zeros")(tA)
    tA = Activation("relu")(tA)

    tA = Dense(10, use_bias=self.HasBias, kernel_initializer="glorot_uniform", bias_initializer="zeros")(tA)
    tA = Softmax()(tA)

    self.Output = tA
    
    self.Model = keras.Model(self.Input, self.Output, name="SimpleCNN")
  # --------------------------------------------------------------------------------------------------------
  def LearningRate(self, p_nEpochNumber):
    if p_nEpochNumber < 10:
      nLR = 0.1
    elif p_nEpochNumber < 20:
      nLR = 0.01
    else:
      nLR = 0.001
    return nLR
  # --------------------------------------------------------------------------------------------------------
# =========================================================================================================================    

Recreate the model. Change and re-run this area of the code that contains all the hyperparameters

In [ ]:
# _____ | Hyperparameters | ______
# // Training \\
MAX_EPOCH                   = 30;
BATCH_SIZE                  = 128    
LEARNING_RATE               = 1e-3;
MOMENTUM                    = 0.9
IS_ACCELERATED_MOMENTUM     = True

sModelFileName = os.path.join("MLModels", "SimpleCNN.keras")
bMustTrain = not os.path.isfile(sModelFileName)

# Train the model if not already trained
if bMustTrain:
  oCNNSimple = CCNNSimple(oDataset.ImageShape)
  oCNNModel = oCNNSimple.Model
  
  oCostFunction = keras.losses.CategoricalCrossentropy()
  oLRSchedule = keras.callbacks.LearningRateScheduler(oCNNSimple.LearningRate) 
  oOptimizer = keras.optimizers.SGD(learning_rate=0.0, momentum=MOMENTUM, nesterov=IS_ACCELERATED_MOMENTUM)   
  oMetric = keras.metrics.CategoricalAccuracy(name="average_accuracy", dtype=None)

  oCNNModel.compile(loss=oCostFunction, optimizer=oOptimizer, metrics=oMetric)

  oCNNModel.train_on_batch(oDataset.TSSamples[0:100, ...], nTSLabelsOnehot[0:100, :]) 
  oCNNModel.summary()
else:
  print("It is already trained!")

Model: "SimpleCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 96)        2688      
_________________________________________________________________
activation (Activation)      (None, 16, 16, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       221440    
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 256)       0 

Train the model and then save the model and training process state to disk. Alternatively load from disk a pre-trained model.

In [ ]:
if bMustTrain:
  # Train the model
  oCNNModel.fit(oDataset.TSSamples, nTSLabelsOnehot, epochs=MAX_EPOCH, batch_size=BATCH_SIZE, callbacks=[oLRSchedule]
                , verbose=1, validation_data=(oDataset.VSSamples, nVSLabelsOnehot))
  if not os.path.exists("MLModels"):
    os.makedirs("MLModels")

  # Saving the state of the model (values of parameters) and the training process (internal values).
  oCNNModel.save(sModelFileName, save_format="h5") 
else:
  # Load the state of the model and the training process
  oCNNModel = keras.models.load_model(sModelFileName)

Epoch 1/30
391/391 [==============================] - 19s 45ms/step - loss: 2.0108 - average_accuracy: 0.2981 - val_loss: 2.4396 - val_average_accuracy: 0.2585
Epoch 2/30
391/391 [==============================] - 16s 41ms/step - loss: 1.4945 - average_accuracy: 0.4624 - val_loss: 1.3840 - val_average_accuracy: 0.4953
Epoch 3/30
391/391 [==============================] - 16s 41ms/step - loss: 1.2357 - average_accuracy: 0.5598 - val_loss: 1.2797 - val_average_accuracy: 0.5294
Epoch 4/30
391/391 [==============================] - 16s 41ms/step - loss: 1.0861 - average_accuracy: 0.6190 - val_loss: 1.0414 - val_average_accuracy: 0.6259
Epoch 5/30
391/391 [==============================] - 16s 40ms/step - loss: 0.9662 - average_accuracy: 0.6627 - val_loss: 1.0434 - val_average_accuracy: 0.6447
Epoch 6/30
391/391 [==============================] - 16s 41ms/step - loss: 0.8770 - average_accuracy: 0.6953 - val_loss: 1.1106 - val_average_accuracy: 0.6261
Epoch 7/30
391/391 [====================

Recall the test samples through a trained model, get the predicted probabilities from the output neurons (softmax activation function) and the maximum out of them

In [ ]:
# Predictions
nPredictedClassProbabilities = oCNNModel.predict(oDataset.VSSamples)
nPredictedClassLabels = np.argmax(nPredictedClassProbabilities, axis=1)

print(nPredictedClassProbabilities.shape)
print(nPredictedClassLabels.shape)
print("")

np.set_printoptions(precision=3, suppress=True) # Pretty print numpy floating point numbers

nTestSampleIndex = 0
print("Ground Truth (one-hot):", nVSLabelsOnehot[nTestSampleIndex,:])
print("Predictions           :", nPredictedClassProbabilities[nTestSampleIndex,:])
print("")

nTestSampleClassLabel = oDataset.VSLabels[nTestSampleIndex]
print("Probability for correct class %s is %.3f" % (oDataset.ClassNames[nTestSampleClassLabel], nPredictedClassProbabilities[nTestSampleIndex, nTestSampleClassLabel]))
nConfusingClassLabel = 5
print("Probability for wrong   class %s is %.3f" % (oDataset.ClassNames[nConfusingClassLabel], nPredictedClassProbabilities[nTestSampleIndex, nConfusingClassLabel]))

(10000, 10)
(10000,)

Ground Truth (one-hot): [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Predictions           : [0.001 0.001 0.005 0.907 0.003 0.024 0.056 0.001 0.001 0.002]

Probability for correct class cat is 0.907
Probability for wrong   class dog is 0.024


Evaluate the model

In [ ]:
# Evaluation
oEvaluator = CEvaluator(oDataset.VSLabels, nPredictedClassLabels)

print(oEvaluator.ConfusionMatrix)
for nIndex,sClassName in enumerate(oDataset.ClassNames):
  print("|__ [%s]  Accuracy:%.4f | Precision:%.4f | F1 Score:%.4f" % (sClassName.rjust(15, " "), oEvaluator.Recall[nIndex], oEvaluator.Precision[nIndex], oEvaluator.F1Score[nIndex]))

print("Average Accuracy: %.4f" % oEvaluator.AverageRecall)
print("Average F1 Score: %.4f" % oEvaluator.AverageF1Score)

[[865  14  29  15  11   2   5  11  32  16]
 [  8 906   2   8   3   1   4   0  18  50]
 [ 48   2 747  40  68  36  38  11   7   3]
 [ 14   4  45 704  51 104  37  25  10   6]
 [ 10   1  43  38 823  21  23  34   5   2]
 [  8   3  24 142  44 722  13  37   2   5]
 [  7   3  25  46  26   6 877   5   3   2]
 [  8   3  18  33  33  32   3 865   0   5]
 [ 36  11   6  14   1   1   4   3 910  14]
 [ 32  49   2   8   2   1   2   8  13 883]]
|__ [       airplane]  Accuracy:0.8650 | Precision:0.8349 | F1 Score:0.8497
|__ [     automobile]  Accuracy:0.9060 | Precision:0.9096 | F1 Score:0.9078
|__ [           bird]  Accuracy:0.7470 | Precision:0.7938 | F1 Score:0.7697
|__ [            cat]  Accuracy:0.7040 | Precision:0.6718 | F1 Score:0.6875
|__ [           deer]  Accuracy:0.8230 | Precision:0.7750 | F1 Score:0.7983
|__ [            dog]  Accuracy:0.7220 | Precision:0.7797 | F1 Score:0.7497
|__ [           frog]  Accuracy:0.8770 | Precision:0.8718 | F1 Score:0.8744
|__ [          horse]  Accuracy:0.865